In [ ]:
#!pip install git+https://github.com/huggingface/diffusers  
#credit to ASOMOZA..  additions are neg prompt embeddings, only the full repo, allowing for guidance manipulation.. and even if it wont make it to video memory, still faster then the chopped up text encoders + q5 gguf i tried .

In [ ]:
import gc, time, os, subprocess
import torch
from transformers import (
    CLIPTextModelWithProjection,
    CLIPTokenizer,
    LlamaForCausalLM,
    PreTrainedTokenizerFast,
    T5EncoderModel,
    T5Tokenizer,
)

from diffusers import AutoencoderKL, HiDreamImagePipeline, HiDreamImageTransformer2DModel, UniPCMultistepScheduler
from diffusers.hooks.group_offloading import apply_group_offloading
from diffusers.image_processor import VaeImageProcessor


repo_id = "HiDream-ai/HiDream-I1-Full"
llama_repo_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
device = torch.device("cuda")
torch_dtype = torch.bfloat16

prompt = """
Candid photo of a dark space fantasy cenobite with nails partially driven into its head, he is sitting at a table inside a 1970s tavern, there are 2 diverse, beautiful human females with serpents for hair, they slither anid hiss. dimly lit rustic atmosphere, other humans stare nervously at this nightmare.

"""
negative_prompt = "cartoon, anime"

width=1344
height=768
guidance_scale=4.5
num_inference_steps=35

def flush(device):
    """Helper function to clear CUDA memory."""
    gc.collect()
    torch.cuda.empty_cache()
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats(device)



# Corrected encode_prompt to manage memory, REVERT Llama embedding to 4D,
# ensure dtype consistency, and use correct dictionary keys for pooled embeddings.
def encode_prompt(
    prompt, negative_prompt, pipeline_repo_id, llama_repo_id, device=device, dtype=torch_dtype
):
    print("Generating embeddings...")
    # Ensure prompts are lists
    prompt = [prompt] if isinstance(prompt, str) else prompt
    # Ensure negative prompt list matches prompt list size
    negative_prompt = [negative_prompt] * len(prompt)

    # --- CLIP 1 ---
    print("  Loading CLIP 1...")
    tokenizer = CLIPTokenizer.from_pretrained(pipeline_repo_id, subfolder="tokenizer")
    # Load to CPU first, then move to device and cast dtype explicitly
    text_encoder = CLIPTextModelWithProjection.from_pretrained(
        pipeline_repo_id, subfolder="text_encoder"
    )
    text_encoder = text_encoder.to(device=device, dtype=dtype) # Explicitly cast

    prompt_embeds = get_clip_prompt_embeds(prompt, tokenizer, text_encoder)
    prompt_embeds_1 = prompt_embeds.clone().detach()
    negative_prompt_embeds = get_clip_prompt_embeds(negative_prompt, tokenizer, text_encoder)
    negative_prompt_embeds_1 = negative_prompt_embeds.clone().detach()

    print("  Unloading CLIP 1...")
    text_encoder.to("cpu") # Move back to CPU to free VRAM
    del prompt_embeds, negative_prompt_embeds
    del tokenizer
    del text_encoder
    flush(device)

    # --- CLIP 2 ---
    print("  Loading CLIP 2...")
    tokenizer = CLIPTokenizer.from_pretrained(pipeline_repo_id, subfolder="tokenizer_2")
    # Load to CPU first, then move to device and cast dtype explicitly
    text_encoder = CLIPTextModelWithProjection.from_pretrained(
        pipeline_repo_id, subfolder="text_encoder_2"
    )
    text_encoder = text_encoder.to(device=device, dtype=dtype) # Explicitly cast

    prompt_embeds = get_clip_prompt_embeds(prompt, tokenizer, text_encoder)
    prompt_embeds_2 = prompt_embeds.clone().detach()
    negative_prompt_embeds = get_clip_prompt_embeds(negative_prompt, tokenizer, text_encoder)
    negative_prompt_embeds_2 = negative_prompt_embeds.clone().detach()

    print("  Unloading CLIP 2...")
    text_encoder.to("cpu") # Move back to CPU to free VRAM
    del prompt_embeds, negative_prompt_embeds
    del tokenizer
    del text_encoder
    flush(device)

    # --- Pooled Embeddings ---
    # Concatenate the output[0] (projected embeddings) from CLIP 1 and CLIP 2
    pooled_prompt_embeds = torch.cat([prompt_embeds_1, prompt_embeds_2], dim=-1)
    negative_pooled_prompt_embeds = torch.cat([negative_prompt_embeds_1, negative_prompt_embeds_2], dim=-1)
    del prompt_embeds_1, prompt_embeds_2, negative_prompt_embeds_1, negative_prompt_embeds_2 # Clean up intermediate CLIP embeddings

    # --- T5 ---
    print("  Loading T5...")
    tokenizer = T5Tokenizer.from_pretrained(pipeline_repo_id, subfolder="tokenizer_3")
    # Load to CPU first, then move to device and cast dtype explicitly
    text_encoder = T5EncoderModel.from_pretrained(
        pipeline_repo_id, subfolder="text_encoder_3"
    )
    text_encoder = text_encoder.to(device=device, dtype=dtype) # Explicitly cast


    # Positive T5
    text_inputs = tokenizer(
        prompt,
        padding="max_length",
        max_length=128,
        truncation=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    text_input_ids = text_inputs.input_ids.to(device)
    attention_mask = text_inputs.attention_mask.to(device) # Ensure attention mask is also on device
    # T5 output [0] is the last hidden state
    prompt_embeds = text_encoder(text_input_ids, attention_mask=attention_mask)[0]
    t5_prompt_embeds = prompt_embeds.clone().detach()

    # Negative T5
    negative_text_inputs = tokenizer(
        negative_prompt,
        padding="max_length",
        max_length=128,
        truncation=True,
        add_special_tokens=True,
        return_tensors="pt",
        )
    negative_input_ids = negative_text_inputs.input_ids.to(device)
    negative_attention_mask = negative_text_inputs.attention_mask.to(device) # Ensure attention mask is also on device
    negative_prompt_embeds = text_encoder(negative_input_ids, attention_mask=negative_attention_mask)[0]
    t5_negative_prompt_embeds = negative_prompt_embeds.clone().detach()

    print("  Unloading T5...")
    text_encoder.to("cpu") # Move back to CPU to free VRAM
    del prompt_embeds, negative_prompt_embeds
    del text_inputs, negative_text_inputs, attention_mask, negative_attention_mask
    del text_encoder
    del tokenizer
    flush(device)

    # --- Llama3 (Text Encoder 4) ---
    print("  Loading Llama3...")
    tokenizer = PreTrainedTokenizerFast.from_pretrained(llama_repo_id)
    tokenizer.pad_token = tokenizer.eos_token # Ensure pad token is set
    # Load to CPU first, then move to device and cast dtype explicitly
    text_encoder = LlamaForCausalLM.from_pretrained(
        llama_repo_id,
        output_hidden_states=True, # Still need this to access hidden states
        output_attentions=False, # Don't need attentions, saves memory during forward pass
    )
    text_encoder = text_encoder.to(device=device, dtype=dtype) # Explicitly cast

    # Positive Llama3
    text_inputs = tokenizer(
        prompt,
        padding="max_length",
        max_length=128,
        truncation=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    text_input_ids = text_inputs.input_ids.to(device)
    attention_mask = text_inputs.attention_mask.to(device) # Ensure attention mask is also on device
    outputs = text_encoder(
        text_input_ids,
        attention_mask=attention_mask,
        output_hidden_states=True, # Request hidden states
        output_attentions=False,
    )
    # *** REVERTED CHANGE: Stack hidden states from layer 1 onwards to get 4D tensor ***
    # This produces shape [num_layers, batch_size, sequence_length, hidden_size]
    llama3_prompt_embeds = torch.stack(outputs.hidden_states[1:], dim=0).clone().detach()


    # Negative Llama3
    negative_text_inputs = tokenizer(
        negative_prompt,
        padding="max_length",
        max_length=128,
        truncation=True,
        add_special_tokens=True,
        return_tensors="pt",
    )
    negative_input_ids = negative_text_inputs.input_ids.to(device)
    negative_attention_mask = negative_text_inputs.attention_mask.to(device) # Ensure attention mask is also on device
    outputs = text_encoder(
        negative_input_ids,
        attention_mask=negative_attention_mask,
        output_hidden_states=True, # Request hidden states
        output_attentions=False,
    )
    # *** REVERTED CHANGE: Stack hidden states from layer 1 onwards to get 4D tensor ***
    # This produces shape [num_layers, batch_size, sequence_length, hidden_size]
    llama3_negative_prompt_embeds = torch.stack(outputs.hidden_states[1:], dim=0).clone().detach()


    print("  Unloading Llama3...")
    text_encoder.to("cpu") # Move back to CPU to free VRAM
    del outputs, text_inputs, negative_text_inputs, attention_mask, negative_attention_mask, text_encoder, tokenizer
    flush(device)

    # --- Assemble Embeddings ---
    # Use the correct keys expected by the pipeline's __call__ method when passing pre-computed embeddings
    embeds = {
        "prompt_embeds_t5": t5_prompt_embeds,
        "prompt_embeds_llama3": llama3_prompt_embeds,
        "pooled_prompt_embeds": pooled_prompt_embeds,
        "negative_prompt_embeds_t5": t5_negative_prompt_embeds,
        "negative_prompt_embeds_llama3": llama3_negative_prompt_embeds,
        "negative_pooled_prompt_embeds": negative_pooled_prompt_embeds,
    }
    print("Embeddings generated.")

    return embeds


def get_clip_prompt_embeds(prompt, tokenizer, text_encoder):
    text_inputs = tokenizer(
        prompt,
        padding="max_length",
        max_length=77,
        truncation=True,
        return_tensors="pt",
    )
    # Ensure input IDs are on the device and are Long type
    text_input_ids = text_inputs.input_ids.to(device, dtype=torch.long)

    # Pass input IDs to the text encoder.
    # The encoder is expected to be already on the correct device and dtype
    # due to the explicit .to(device=device, dtype=dtype) call in encode_prompt.
    outputs = text_encoder(text_input_ids, output_hidden_states=True)

    # CLIPTextModelWithProjection output is typically a tuple: (text_embeds, pooled_output, hidden_states...)
    # text_embeds is the result of text_projection(pooled_output)
    # We take outputs[0], the final projected embedding as in your original code.
    prompt_embeds = outputs[0]

    return prompt_embeds


def denoise(embeddings, device=device, dtype=torch_dtype):
    print("Loading Denoising components...")
    scheduler = UniPCMultistepScheduler(
        flow_shift=3.0,
        prediction_type="flow_prediction",
        use_flow_sigmas=True,
    )

    transformer = HiDreamImageTransformer2DModel.from_pretrained(
        "HiDream-ai/HiDream-I1-Full", subfolder="transformer", torch_dtype=torch_dtype
    )

    # Apply offloading to the transformer
    transformer.enable_layerwise_casting(storage_dtype=torch.float8_e4m3fn, compute_dtype=torch_dtype)
    apply_group_offloading(
        transformer,
        onload_device=device,
        offload_device=torch.device("cpu"),
        offload_type="leaf_level",
        use_stream=True,
        low_cpu_mem_usage=True,
    )

    # Load VAE only for scaling factor within the pipeline object initialization
    # It will be loaded again later for actual decoding outside the pipeline call.
    # Load to CPU initially, it seems the pipeline constructor might not need it on device immediately
    vae_init = AutoencoderKL.from_pretrained(repo_id, subfolder="vae", torch_dtype=torch_dtype)


    print("Instantiating pipeline (without text encoders)...")
    # Instantiate the pipeline by passing only the components it needs for the diffusion process
    # and which will receive the pre-computed embeddings.
    # We explicitly pass None for text encoders and tokenizers as we're using pre-computed embeddings.
    # REMOVED torch_dtype=torch_dtype from THIS constructor call - this was the fix for the last error
    pipe = HiDreamImagePipeline(
        vae=vae_init, # VAE is needed for scaling
        transformer=transformer,
        scheduler=scheduler,
        text_encoder=None,
        tokenizer=None,
        text_encoder_2=None,
        tokenizer_2=None,
        text_encoder_3=None,
        tokenizer_3=None,
        text_encoder_4=None, # Ensure Text Encoder 4 (Llama) is also None
        tokenizer_4=None,
        # Removed: torch_dtype=torch_dtype,
    )

    # Move pipeline components (transformer, scheduler, and potentially VAE_init) to device
    # Note: The Transformer will handle its own offloading due to apply_group_offloading
    pipe.to(device)

    print("Running pipeline...")
    # Pass the pre-computed embeddings to the pipeline
    latents = pipe(
        prompt_embeds_t5=embeddings["prompt_embeds_t5"],
        prompt_embeds_llama3=embeddings["prompt_embeds_llama3"],
        pooled_prompt_embeds=embeddings["pooled_prompt_embeds"],
        negative_prompt_embeds_t5=embeddings["negative_prompt_embeds_t5"],
        negative_prompt_embeds_llama3=embeddings["negative_prompt_embeds_llama3"],
        negative_pooled_prompt_embeds=embeddings["negative_pooled_prompt_embeds"],
        height=height,
        width=width,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device).manual_seed(0), # Generator on device
        output_type="latent", # Output latents as in your original code
        return_dict=False,
    )[0]

    print("Denoising complete.")
    # Clean up pipeline components if necessary (though they might offload themselves)
    del pipe, transformer, scheduler, vae_init
    flush(device)

    return latents

# --- Main Execution ---
time_gen = time.time()

with torch.no_grad():
    # Generate embeddings using the memory-managed function
    embeddings = encode_prompt(prompt, negative_prompt, repo_id, llama_repo_id, device=device, dtype=torch_dtype)


# Denoise using the pre-computed latents
latents = denoise(embeddings, device=device, dtype=torch_dtype)

print("Loading VAE for decoding...")
# Load VAE separately for decoding, as denoise outputs latents
vae = AutoencoderKL.from_pretrained(repo_id, subfolder="vae", torch_dtype=torch_dtype).to(device)

# Apply scaling factor and shift factor before decoding
latents = (latents / vae.config.scaling_factor) + vae.config.shift_factor

with torch.no_grad():
    print("Decoding latents...")
    image = vae.decode(latents, return_dict=False)[0]

print("Post-processing image...")
# The HiDream VAE scaling factor for the image processor is typically vae.config.scaling_factor * 8
# based on the model architecture (latent size is image_size / 8).
# The correct calculation is vae.config.scaling_factor * 2^(num_downsampling_blocks)
vae_scale_factor = vae.config.scaling_factor * (2 ** (len(vae.config.block_out_channels) - 1))
image_processor = VaeImageProcessor(vae_scale_factor=vae_scale_factor) # Use the derived scale factor
image = image_processor.postprocess(image, output_type="pil")[0]


# Clean up VAE after decoding
del vae
flush(device)


filename = f"hidream_cfg{guidance_scale}_steps_{num_inference_steps}_{str(int(time.time()))}.png"

print("\n--- NVIDIA-SMI Stats ---")
try:
    result = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used,temperature.gpu,utilization.gpu', '--format=csv,noheader'], encoding='utf-8', timeout=10.0)
    print(result.strip())
except FileNotFoundError:
    print("nvidia-smi not found. Cannot retrieve GPU stats.")
except subprocess.TimeoutExpired:
    print("nvidia-smi timed out.")
except Exception as e:
    print(f"Error running nvidia-smi: {e}")
print("----------------------\n")

image.save(filename)
print(f"Generated image saved as {filename}")
os.startfile(filename)
print(f"   ... Total generation time: {time.time() - time_gen:.2f} secs")